# Привет!

Перед началом прохождения убедитесь, что все настроено как надо. Следующая ячейка должна выполниться без ошибок:

После прохождения не забудьте **перезапустить ядро** (**restart the kernel**) (в панели меню выберите Kernel $\rightarrow$ Restart) и затем **запуск всех ячеек** (**run all cells**) (в панели меню выберите Cell $\rightarrow$ Run All).

Убедитесь, что заменили все такие места 
```python
# YOUR CODE HERE
raise NotImplementedError()
```
и
```
"YOUR ANSWER HERE"
```
на свои решения и ответы.

Если не установлены какие-то пакеты, то посмотрите нужные версии в ноутбуках по регрессии.

Для прорешивания используйте среды с поддержкой формата Jupyter: [Google Colab](https://colab.research.google.com/), [Jupyter Notebook (Web)](https://jupyter.org/), [PyCharm](https://www.jetbrains.com/ru-ru/pycharm/), [DataSpell](https://www.jetbrains.com/ru-ru/dataspell/) или [VSCode](https://code.visualstudio.com/). Работа с этим форматом в обычном текстовом редакторе может сыграть злую шутку.

Не забывайте о следующих особенностях:
- 😉 Помимо указанных тестов после задания существуют и скрытые тесты, поэтому внимательнее читайте задание
- 😎 Обязательно проверяйте выполнимость на тестах, для проверки перезапустите ядро и запустите все ячейки заново!
- 😂 После скачивания ноутбуков достаточно заполнить места для ответов и отправить ноутбук обратно (сам ноутбук, ни в архиве, ни ссылкой на GitHub ни как-то еще принимать не будем, загружаете сам файл)
- 😈 Не рекомендуем удалять исходные ячейки, так как из-за этого могут не начислиться баллы =(
  - 👼 Если таки такое случилось, скачайте заново шаблон и перенесите в него свои решения
- 😛 Свой код пишите только в ячейках для решений - любые дописывания в остальных ячейках (с текстом или тестами) могут не попасть в проверку (например, `import` вне ячейки для решения)


---

Это третья контрольная работа с некоторыми вопросами по теме классификации в машинном обучении!

In [1]:
import numpy as np
print(f"numpy version: {np.__version__}")
import pandas as pd
print(f"pandas version: {pd.__version__}")
# Just to check that it exists
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")

from sklearn.datasets import load_iris, load_linnerud
from sklearn.metrics import classification_report, confusion_matrix

from typing import Union

RANDOM_SEED = 123
np.random.seed(RANDOM_SEED)

numpy version: 1.26.4
pandas version: 2.2.2
scikit-learn version: 1.4.2


### Задание 1

У вас имеется следующий код и разработчик дает следующее описание:

"Я сделал код для датасета linnerud, который выбирает примеры со значением прыжков меньше 80. Написал как смог =)"

```python
valid_data = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if j == 2:
            row = data.iloc[i] 
            value = row[2]
            if value < 80:
                valid_data.append(row.name)
```

Ваша задача - написать код, который выбирает примеры с количеством приседаний от 120 до 250 (не включительно). Для реализации необходимо использовать pandas.

In [2]:
def extract_average_situps_indexes(data: pd.DataFrame) -> list:
    extract_situps = data.iloc[:, 1]
    filter_situps = extract_situps[(extract_situps > 120) & (extract_situps < 250)].index.tolist()
    return filter_situps


In [3]:
features, target = load_linnerud(return_X_y=True, as_frame=True)

indexes = extract_average_situps_indexes(features)
np.testing.assert_array_equal(indexes, [0, 4, 7, 8, 11, 12, 15, 17, 18])


### Задание 2

Напишите класс классификатора, который проверяет значение задаваемой колонки (индекса по порядку) (anomaly_col_idx) и для примеров со значением больше заданного порога (threshold) **по абсолютному значению** записывает в предсказание "Anomaly!". Для предсказания используйте LogisticRegression и не забудьте зафиксировать random_state (для этого есть RANDOM_SEED переменная)!


In [4]:
from sklearn.linear_model import LogisticRegression


class ClassifierWithAnomalyDetection:
    def __init__(self, anomaly_col_idx: int, threshold: float):
        self.anomaly_col_idx = anomaly_col_idx
        self.threshold = threshold
        self.logreg = LogisticRegression(random_state=RANDOM_SEED)
        

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        self.logreg.fit(X_train, y_train)
        

    def predict(self, X_test: pd.DataFrame):
        y_pred = self.logreg.predict(X_test)
        anomaly_col = X_test.iloc[:, self.anomaly_col_idx]
        for i in range(len(anomaly_col)):
            if abs(anomaly_col.iloc[i]) > self.threshold:
                y_pred[i] = 'Anomaly!'
        return y_pred
        

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Prepare data for testing
X_data, y_data = make_classification(
    n_samples=100, 
    n_features=2, 
    n_redundant=0,
    scale=[1, 10],
    random_state=RANDOM_SEED)

train_columns = ["Feature_1", "Feature_2"]
target_column = "target"

df = pd.DataFrame(X_data, columns=train_columns)
df[target_column] = pd.Series(y_data).map({0: "bad", 1: "good"})

train_df, test_df = train_test_split(df, test_size=0.3, random_state=RANDOM_SEED)

classifier = ClassifierWithAnomalyDetection(anomaly_col_idx=0, threshold=18)
    
classifier.fit(train_df[train_columns], train_df[target_column])
y_pred = classifier.predict(test_df[train_columns])
y_true = test_df[target_column]

assert "Anomaly!" in y_pred
assert "bad" in y_pred
assert "good" in y_pred

np.testing.assert_equal(len(y_pred), len(test_df))


### Задание 3

Перепешите реализацию кода с использованием pandas:

```python
df_1_data = df_main.values
df_2_data = df_noise.values

new_column = [] 
for i in range(df_1_data.shape[0]):
    for j in range(df_1_data.shape[1]):
        if j == 0:
            for k in range(df_2_data.shape[0]):
                for m in range(df_2_data.shape[1]):
                    if m == 0:
                        if df_1_data[i][j] == df_2_data[k][m]:
                            for n in range(df_2_data.shape[1]):
                                if n == 1:
                                    new_column.append(df_2_data[k][n])
df_main = df_main.copy()
df_main['has_noise'] = new_column

return df_main
```

In [6]:
def append_noise_column(df_main: pd.DataFrame, df_noise: pd.DataFrame) -> pd.DataFrame:
    df_with_pandas = df_main.merge(df_noise, on='id', how='left')
    return df_with_pandas
    

In [7]:
df_main = pd.DataFrame(
    [
        [1, 1.5], [1, 2.3], 
        [2, 0.7], [2, 1.6], [2, 3.4], 
        [3, 0.5], [3, 0.5], [3, 0.5]
    ], 
    columns=["id", "value"]
)
df_noise = pd.DataFrame(
    [
        [1, True], [2, False], [3, False]
    ], 
    columns=["id", "has_noise"]
)

df_with_noise = append_noise_column(df_main, df_noise)

np.testing.assert_array_equal(df_with_noise.columns, ["id", "value", "has_noise"])

np.testing.assert_equal(df_with_noise.loc[0, "has_noise"], True)
np.testing.assert_equal(df_with_noise.loc[4, "has_noise"], False)
np.testing.assert_equal(df_with_noise.loc[7, "has_noise"], False)


### Задание 4

Найдите индексы строк, где колонка делится на n_div без остатка. Например, значения wine_serving на 3.

In [8]:
def get_n_divisible_indexes(df: pd.DataFrame, column_name: str, n_div: int) -> list:
    divisible_indexes = []

    for i in range(len(df)):
        val_div = df.loc[i, column_name]
        if val_div % n_div == 0:
            divisible_indexes.append(i)
    return divisible_indexes    

In [9]:
features, target = load_linnerud(return_X_y=True, as_frame=True)

indexes = get_n_divisible_indexes(features, "Jumps", 5)
np.testing.assert_array_equal(indexes, [0,  1,  7,  8,  9, 11, 12, 13, 15, 16, 17])


### Задание 5

Напишите функцию, которая строит confusion matrix (true ~ rows, pred ~ cols) по заданным классам по осям. Остальные классы скидываются в объединенный класс "Другое".

Например, имеем:

```python
y_true = [0, 1, 1, 2, 2, 3, 1, 1]
y_pred = [1, 1, 1, 3, 2, 3, 1, 0]
```

Мы задаем целевые классы

```python
true_classes = [0, 1]
pred_classes = [1, 2]
```

Должна получиться матрица (DataFrame) с соответствующими именами строк и колонок. Порядок тоже должен сохраняться по задаваемому списку классов.

|    |  1 | 2  | Другое |
|---:|:--:|:--:|:--:  |
| 0  | 1  | 0  |    0    |
| 1  | 3  | 0  |   1     |
|Другое| 0 | 1   |   2     |


In [10]:
def custom_confusion_matrix(y_true: list, y_pred: list, true_classes: list, pred_classes:list) -> pd.DataFrame:
    true_classes_with_other = true_classes + ["Другое"]
    pred_classes_with_other = pred_classes + ["Другое"]

    cm = pd.DataFrame(0, index=true_classes_with_other, columns=pred_classes_with_other)

    for true, pred in zip(y_true, y_pred):
        if true in true_classes:
            true_class = true
        else:
            true_class = 'Другое'

        if pred in pred_classes:
            pred_class = pred
        else:
            pred_class = 'Другое'
            
        cm.loc[true_class, pred_class] += 1
    return cm    

In [11]:
y_true = [0, 1, 1, 2, 2, 3, 1, 1]
y_pred = [1, 1, 1, 3, 2, 3, 1, 0]

true_classes = [0, 1]
pred_classes = [1, 2]

conf = custom_confusion_matrix(y_true, y_pred, true_classes, pred_classes)
assert isinstance(conf, pd.DataFrame)
np.testing.assert_array_equal(conf.columns, np.array([1, 2, "Другое"], dtype=object))
np.testing.assert_array_equal(conf.index, np.array([0, 1, "Другое"], dtype=object))
np.testing.assert_equal(conf.loc[1, 1], 3)
np.testing.assert_equal(conf.loc["Другое", 2], 1)

In [12]:
# Another test
y_true = [0, 1, 1, 2, 2, 3, 1, 1, 2, 1, 0]
y_pred = [1, 1, 1, 3, 2, 3, 1, 0, 4, 1, 2]

true_classes = [3, 0]
pred_classes = [1, 2]

conf = custom_confusion_matrix(y_true, y_pred, true_classes, pred_classes)
np.testing.assert_array_equal(conf, [[0, 0, 1], [1, 1, 0], [4, 1, 3]]) 



### Задание 6

Напишите функцию, которая принимает вектор предсказаний и возвращает индексы примеров, в которых модель совершила ошибки. Обратите внимание, что входной вектор может быть как обычным list или tuple, так и pd.Series. Вернуть необходимо индексы (index) из исходного y_true.

In [13]:
def get_error_indexes(y_true: pd.Series, y_pred: Union[list, tuple, pd.Series]) -> pd.Index:
    if not isinstance(y_pred, pd.Series):
        y_pred = pd.Series(y_pred, index=y_true.index)
    else:
        y_pred = y_pred.set_axis(y_true.index)  

    error_idx = y_true.index[y_true != y_pred]
    return error_idx

In [14]:
X, y = load_iris(as_frame=True, return_X_y=True)

y_test = y.iloc[50:60]
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 1, 1]

err_idxs = get_error_indexes(y_test, y_pred)
np.testing.assert_array_equal(err_idxs, [50, 51, 55, 56, 57])


In [15]:
X, y = load_iris(as_frame=True, return_X_y=True)

y_test = y.iloc[70:80]
y_pred = pd.Series([0, 1, 0, 1, 0, 1, 1, 0, 0, 1])

err_idxs = get_error_indexes(y_test, y_pred)

np.testing.assert_array_equal(err_idxs, [70, 72, 74, 77, 78])


### Задание 7

Напишите функцию, которая по переданным `y_true, y_pred` строит стандартный `classification_report`, вычисляет `confusion_matrix` и обогащает данные из `classification_report` столбцом `specificity`. Столбец необходимо посчитать для каждого класса, а также `macro` и `weighted` значения. 
Используйте готовые реализации там, где это возможно.

In [16]:
from sklearn.metrics import classification_report, confusion_matrix


def extend_classification_report(y_true: np.ndarray, y_pred: np.ndarray, class_labels) -> pd.DataFrame:
    dict_report = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
    report = pd.DataFrame(dict_report).transpose()

    cm = confusion_matrix(y_true, y_pred, labels=class_labels)

    specificity = []
    for i, label in enumerate(class_labels):
        tn = np.sum(cm) - (np.sum(cm[:, i]) + np.sum(cm[i, :]) - cm[i, i])
        fp = np.sum(cm[:, i]) - cm[i, i]
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        specificity.append(spec)

    for i, label in enumerate(class_labels):
        report.loc[label, 'specificity'] = specificity[i]

    macro_avg_metrics = np.mean(specificity)
    report.loc['macro avg', 'specificity'] = macro_avg_metrics

    class_counts = np.sum(cm, axis=1)
    weighted_avg_metrics = np.average(specificity, weights=class_counts)
    report.loc['weighted avg', 'specificity'] = weighted_avg_metrics

    expected_index = class_labels + ['accuracy', 'macro avg', 'weighted avg']
    report = report.reindex(expected_index)
    return report


In [17]:
class_labels = ['yellow', 'red', 'white']
y_true = ['yellow', 'white', 'white', 'red', 'red', 'white', 'white', 'white', 'yellow', 'yellow', 'red', 'red']
y_pred = ['white', 'white', 'white', 'red', 'white', 'yellow', 'red', 'white', 'red', 'red', 'red', 'white']

df = extend_classification_report(y_true, y_pred, class_labels)

np.testing.assert_array_equal(df.index, ['yellow', 'red', 'white', 'accuracy', 'macro avg', 'weighted avg'])

np.testing.assert_almost_equal(df['specificity']['yellow'], 0.833)
np.testing.assert_almost_equal(df['specificity']['red'], 0.5)
np.testing.assert_almost_equal(df['specificity']['white'], 0.4)

np.testing.assert_almost_equal(df['specificity']['macro avg'], 0.578)
np.testing.assert_almost_equal(df['specificity']['weighted avg'], 0.542)


AssertionError: 
Arrays are not almost equal to 7 decimals
 ACTUAL: 0.8888888888888888
 DESIRED: 0.833